In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin


In [3]:
base_url="https://openmusictheory.github.io/"
TOC_URL = "https://openmusictheory.github.io/contents.html"

def get_chapter_links():
    res=requests.get(TOC_URL)
    soup=BeautifulSoup(res.text,"html.parser")

    links=[]
    for a in soup.select("a[href]"):
        href=a["href"]
        if href.endswith("html") and not href.startswith("http"):
            full_url=urljoin(base_url,href)
            links.append(full_url)

    return links

all_links=get_chapter_links()
print(f"FOUND {len(all_links)}'s of links")

FOUND 98's of links


In [4]:
def get_text(url):
    res=requests.get(url)
    soup=BeautifulSoup(res.text,"html.parser")

    main=soup.find("article")
    if not main:
        main=soup.find("body")

    return main.get_text("\n",strip=True)

In [6]:
import os

# Ensure the directory exists
os.makedirs("omt_texts", exist_ok=True)

for url in all_links:
    try:
        text = get_text(url)
        filename = url.split("/")[-1].replace(".html", "") + ".txt"
        filepath = os.path.join("omt_texts", filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(text)
    except Exception as e:
        print(f"Error scraping {url}: {e}")
